In [1]:
%run binarizers.ipynb
%run data.ipynb
%run models.ipynb

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = Net().to(device)

In [3]:
epochs = 100
lr = 1e-3
rho = 0.5
optimizer = optim.Adam(net.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train(epochs):
    net.train()
    
    for epoch in range(epochs):
        for idx, (data, target) in enumerate(train_loader):
            if device == 'cuda':
                data, target = data.cuda(), target.cuda()
            data, target = Variable(data), Variable(target)

            optimizer.zero_grad()
            output = net(data)
            loss = criterion(output, target)

            if epoch % 40 == 0:
                optimizer.param_groups[0]['lr'] = optimizer.param_groups[0]['lr']*0.1

            loss.backward()
            
            for p in list(net.parameters()):
                if hasattr(p,'org'):
                    p.data.copy_(p.org)
            
            optimizer.step()
            
            for p in list(net.parameters()):
                if hasattr(p,'org'):
                    p.org.copy_(p.data.clamp_(-1,1))
            
            with torch.no_grad():
                if idx % 10 == 0:
                    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, idx * len(data), len(train_loader.dataset),
                        100. * idx / len(train_loader), loss.item()))
        test()

def test():
    net.eval()
    test_loss, correct = 0, 0
    
    for data, target in test_loader:
        if device == 'cuda':
            data, target = data.cuda(), target.cuda()
            
        with torch.no_grad():
            data, target = Variable(data), Variable(target)
            
            output = net(data)
            
            test_loss += criterion(output, target).item()
            
            pred = torch.max(output, dim=1)[1]
                                    
            correct += (pred == target).sum()
    
    test_loss /= len(test_loader.dataset)
    
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

train(epochs)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:54: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.375952
Train Epoch: 0 [640/60000 (1%)]	Loss: 2.337902
Train Epoch: 0 [1280/60000 (2%)]	Loss: 2.412356
Train Epoch: 0 [1920/60000 (3%)]	Loss: 2.337389
Train Epoch: 0 [2560/60000 (4%)]	Loss: 2.372465
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.445740
Train Epoch: 0 [3840/60000 (6%)]	Loss: 2.357227
Train Epoch: 0 [4480/60000 (7%)]	Loss: 2.408090
Train Epoch: 0 [5120/60000 (9%)]	Loss: 2.417967
Train Epoch: 0 [5760/60000 (10%)]	Loss: 2.390544
Train Epoch: 0 [6400/60000 (11%)]	Loss: 2.359569
Train Epoch: 0 [7040/60000 (12%)]	Loss: 2.400205
Train Epoch: 0 [7680/60000 (13%)]	Loss: 2.318829
Train Epoch: 0 [8320/60000 (14%)]	Loss: 2.351899
Train Epoch: 0 [8960/60000 (15%)]	Loss: 2.361306
Train Epoch: 0 [9600/60000 (16%)]	Loss: 2.337667
Train Epoch: 0 [10240/60000 (17%)]	Loss: 2.427793
Train Epoch: 0 [10880/60000 (18%)]	Loss: 2.357608
Train Epoch: 0 [11520/60000 (19%)]	Loss: 2.375571
Train Epoch: 0 [12160/60000 (20%)]	Loss: 2.394739
Train Epoch: 0 [12800/60000 (